In [60]:
import sklearn
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [61]:
df = pd.read_csv('clean_house_data.csv')
df.head()

,id,Zip,Price,Price of square meter,Building Cond. values,Number of facades,Number of rooms,Living area,Furnished,Kitchen values,Surface of the land,Primary energy consumption,Energy_classes,Terrace,Terrace surface,Garden,Garden surface,Open fire,Swimming pool
0,10597213,4000,380000,1759.26,4.0,4.0,5.0,216.0,0.0,1.0,2100.0,335.0,4,0,0.0,1,1800.0,1.0,0.0
1,10594958,4400,315000,1730.77,2.0,4.0,3.0,182.0,0.0,2.0,1580.0,480.0,3,0,0.0,1,1000.0,0.0,0.0
2,10595263,1420,750000,2737.23,4.0,4.0,3.0,274.0,0.0,3.0,1567.0,157.0,6,1,62.0,1,1428.0,1.0,0.0
3,10593759,4020,179000,1598.21,3.0,2.0,3.0,112.0,0.0,1.0,135.0,304.0,4,1,100.0,1,35.0,0.0,0.0
4,10593565,9600,209000,1583.33,3.0,2.0,3.0,132.0,0.0,3.0,180.0,252.0,5,1,20.0,1,80.0,0.0,0.0


In [62]:
y = df[['Price']]
x = df[['id','Zip',
       #'Building Cond. values',
       'Number of facades',
       'Number of rooms','Living area','Furnished',
       'Kitchen values','Surface of the land',
       'Energy_classes',
       'Terrace surface',
       'Garden surface','Open fire','Swimming pool']]

In [63]:
X.isna().sum()

id                       0
Zip                      0
Number of rooms         61
Living area              0
Furnished                0
Surface of the land    486
Energy_classes           0
Terrace                  0
Garden                   0
Open fire                0
Swimming pool            0
dtype: int64

convert df to numpy array ?

In [64]:
# .to_numpy()

In [65]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=101) 

In [66]:
from sklearn.linear_model import LinearRegression 

lm = LinearRegression() 

lm.fit(X_train,y_train) 

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
# Create a logistic regression model

In [ ]:
# Train the model

In [ ]:
# Make predictions on the test set

In [ ]:
# Evaluate the model